In [5]:
# https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [75]:
import numpy as np
from sklearn.model_selection import train_test_split

In [45]:
def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_label in classes:
            p = [row[-1] for row in group].count(class_label) / size
            score += p * p 
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [46]:
gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1])

0.5

In [47]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else: 
            right.append(row)
    return left, right

In [49]:
def get_split(dataset):
    class_values = list(set([row[-1] for row in dataset]))
    b_index, b_value, b_score, b_groups = 999,999,999, None
    for index in range(len(dataset[0]) - 1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
                
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [50]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]

In [51]:
split = get_split(dataset)

In [52]:
split

{'index': 0,
 'value': 6.642287351,
 'groups': ([[2.771244718, 1.784783929, 0],
   [1.728571309, 1.169761413, 0],
   [3.678319846, 2.81281357, 0],
   [3.961043357, 2.61995032, 0],
   [2.999208922, 2.209014212, 0]],
  [[7.497545867, 3.162953546, 1],
   [9.00220326, 3.339047188, 1],
   [7.444542326, 0.476683375, 1],
   [10.12493903, 3.234550982, 1],
   [6.642287351, 3.319983761, 1]])}

In [53]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [54]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

In [55]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

In [60]:
tree = build_tree(dataset, 1,1)

In [61]:
tree

{'index': 0, 'value': 6.642287351, 'left': 0, 'right': 1}

In [59]:
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [74]:
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = []
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return predictions

In [73]:
for row in dataset:
    pred = predict(tree, row)
    print(f"Actual = {row[-1]}. Pred: {pred}")

Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 0. Pred: 0
Actual = 1. Pred: 1
Actual = 1. Pred: 1
Actual = 1. Pred: 1
Actual = 1. Pred: 1
Actual = 1. Pred: 1


In [67]:
bank_data = np.genfromtxt('data_banknote_authentication.txt', delimiter=',')

In [71]:
bank_data[-1]

array([-2.5419 , -0.65804,  2.6842 ,  1.1952 ,  1.     ])

In [77]:
pred = decision_tree(bank_data, bank_data, 5, 10)

In [78]:
len(pred) == len(bank_data)

True

In [86]:
correct = 0
for i in range(len(pred)):
    if pred[i] == bank_data[i][-1]:
        correct+=1
    
correct / len(bank_data)

0.9839650145772595